In [1]:
from gensim.models import Word2Vec
from tensorflow.python.keras.models import Sequential, Model, load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout, Input, Concatenate

import sys
sys.path.insert(1, '../utils')
from NLP_little_helpers import *
little_helpers()

Using TensorFlow backend.
[nltk_data] Downloading package brown to /home/felipe/nltk_data...
[nltk_data]   Package brown is already up-to-date!


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, predict_tags, tagged_n_grams, unknown_words_X, check_and_predict


### Pre-processing data

In [2]:
# Movie conversation cleaned and POS tagged data
with open('clean_tagged_data.json', 'r') as outfile:
    data = json.load(outfile)
    
sent, tags = zip(*data['tags'])

print('Sentences/Tags data length:', len(sent), len(tags))
print(sent[:3])
print(tags[:3])

Sentences/Tags data length: 289401 289401
('<BOS> they do not <EOS>', '<BOS> they do to <EOS>', '<BOS> i hope so <EOS>')
('<start> PPSS DO * <end>', '<start> PPSS DO TO <end>', '<start> NN NN RB <end>')


In [4]:
sent = sent[:20000]
tags = tags[:20000]

In [5]:
# Set the number of n-grams to build sequences
n = 5

# maximum length of sequence is n-1 since the last word will be the target (y)
max_length = n-1

In [6]:
# Tokenizing and getting n-gram sequences

tk_text = Tokenizer(filters=[])
tk_text.fit_on_texts(sent)
enc_sentences = tk_text.texts_to_sequences(sent)

tk_tags = Tokenizer(filters=[])
tk_tags.fit_on_texts(tags)
enc_tagged = tk_tags.texts_to_sequences(tags)

tagged_sent = zip(enc_sentences, enc_tagged)

text_grams, tag_grams = tagged_n_grams(tagged_sent, n)

X, y, X_rev, y_rev = text_grams
X_tag, y_tag, X_tag_rev, y_tag_rev = tag_grams

vocab_size = len(tk_text.word_index)+1
tags_size = len(tk_tags.word_index)+1
print('Vocabulary Size: %d' % vocab_size)

Total Sequences: 779764
Vocabulary Size: 12133


In [7]:
# splitting sentences for word2vec
splt_sent = [s.lower().split() for s in sent]
splt_sent[:2]

[['<bos>', 'they', 'do', 'not', '<eos>'],
 ['<bos>', 'they', 'do', 'to', '<eos>']]

In [ ]:
# embedding words with word2vec (skip-gram)
embedded = Word2Vec(splt_sent, sg=1, min_count=0)

In [ ]:
# creating embedding_matrix to associate embedding with encoded sequences
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tk_text.word_index.items():
    embedding_matrix[i] = embedded.wv[word]

In [13]:
with open('text_tokenizer.pkl', 'wb') as f:
    pickle.dump(tk_text, f)
    
with open('tags_tokenizer.pkl', 'wb') as f:
    pickle.dump(tk_tags, f)
    
with open('max_length.pkl', 'wb') as f:
    pickle.dump(max_length, f)
    
with open('w2v.pkl', 'wb') as f:
    pickle.dump(embedded, f)

### Modeling and Training for OOV Prediction

In [ ]:
# using n-grams of sentences, tags, reversed sentences and reversed tags
# using word2vec embedding for sentences and standard keras embedding for tags
# concatenating each model to get a more reliable prediction 

text_input = Input((max_length,))
model_text = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(text_input)
model_text = Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.1))(model_text)

tags_input = Input((max_length,))
model_tags = Embedding(vocab_size, 100)(tags_input)
model_tags = Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.1))(model_tags)

concatenate_text_tags = Concatenate()([model_text, model_tags])

rev_text_input = Input((max_length,))
rev_model_text = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(rev_text_input)
rev_model_text = Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.1))(rev_model_text)

rev_tags_input = Input((max_length,))
rev_model_tags = Embedding(vocab_size, 100)(rev_tags_input)
rev_model_tags = Bidirectional(LSTM(16, dropout=0.2, recurrent_dropout=0.1))(rev_model_tags)

rev_concatenate_text_tags = Concatenate()([rev_model_text, rev_model_tags])

concatenate = Concatenate()([concatenate_text_tags, rev_concatenate_text_tags])

result = Dense(vocab_size+1, activation='softmax')(concatenate) # check this

model = Model(inputs=[text_input, tags_input, rev_text_input, rev_tags_input], outputs=result)

model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001),
                       metrics=['accuracy'])

model.fit([X, X_tag, X_rev, X_tag_rev], y, batch_size=128, epochs=5, verbose=1, shuffle=True, validation_split=0.2)

model.save('model_full_concat.h5')